In [3]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import datetime
import warnings 
warnings.filterwarnings(action= 'ignore')

In [4]:
# Import file
df = pd.read_csv('https://raw.githubusercontent.com/OscarG-DataSci/ADS-599B/main/Data%20Folder/new_elpac.csv')
df.head()#check the head

,School_deID,GradeLevel,StudentGender,StudentEthnicity,Special_Education,Homeless,SocioEconomically,TestDayName,OverallScore,OverallLevel,ExpectedAttendanceDays,DaysAttended,EnrolledPct,GradeAttendedPct,TeacherGender,TeacherTotalYearsOfService,TeacherEthnicity,OverallScoreStd,TotalAssessments,TestInstance,Growth
0,3,1,0,4,0,1,1,6,1503,4,180.0,175.0,1.0,1.972222,0,4,3,0.740042,3,1,0
1,3,2,0,4,0,1,1,5,1549,4,180.0,175.0,1.0,2.972222,0,22,3,0.725455,3,2,0
2,3,3,0,4,0,1,1,5,1466,2,180.0,177.0,1.0,3.983333,0,25,3,0.605364,3,3,-2
3,5,4,1,4,0,0,1,0,1535,3,180.0,168.0,1.0,4.933333,0,2,5,0.662651,1,1,0
4,6,3,1,4,0,0,1,6,1412,1,180.0,179.0,1.0,3.994444,0,2,3,0.501916,3,1,0


In [5]:
from sklearn.model_selection import train_test_split

X = df.loc[ : , df.columns != 'OverallLevel']
y = df[['OverallLevel']]
#split 70% train 30% test
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=700, test_size=0.30)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(6622, 20) (6622, 1) (2838, 20) (2838, 1)


In [7]:
#create a new dataframe, will convert the Y variable to categorical
df.X = df.copy()
df.X.shape

(9460, 21)

In [22]:
# convert the OverallLevel to categorical 
df['results'] = 'na'
df.X.loc[(df.X.OverallLevel== 4),'results']= 'pass'
df.X.loc[(df.X.OverallLevel== 3),'results']= 'fair'
df.X.loc[(df.X.OverallLevel<= 2) ,'results']= 'poor'
df.X.drop(columns=['OverallLevel'])#drop the overall column

,School_deID,GradeLevel,StudentGender,StudentEthnicity,Special_Education,Homeless,SocioEconomically,TestDayName,OverallScore,ExpectedAttendanceDays,DaysAttended,EnrolledPct,GradeAttendedPct,TeacherGender,TeacherTotalYearsOfService,TeacherEthnicity,OverallScoreStd,TotalAssessments,TestInstance,Growth,results
0,3,1,0,4,0,1,1,6,1503,180.0,175.0,1.000000,1.972222,0,4,3,0.740042,3,1,0,pass
1,3,2,0,4,0,1,1,5,1549,180.0,175.0,1.000000,2.972222,0,22,3,0.725455,3,2,0,pass
2,3,3,0,4,0,1,1,5,1466,180.0,177.0,1.000000,3.983333,0,25,3,0.605364,3,3,-2,poor
3,5,4,1,4,0,0,1,0,1535,180.0,168.0,1.000000,4.933333,0,2,5,0.662651,1,1,0,fair
4,6,3,1,4,0,0,1,6,1412,180.0,179.0,1.000000,3.994444,0,2,3,0.501916,3,1,0,poor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9455,6,1,1,4,1,0,0,0,1397,170.0,140.0,0.944444,1.823529,0,7,7,0.517820,1,1,0,poor
9456,4,4,0,4,0,1,1,6,1362,130.0,105.0,0.722222,4.807692,0,13,3,0.364888,1,1,0,poor
9457,9,0,0,4,0,0,1,0,1523,130.0,96.0,0.722222,0.738462,0,13,3,0.678182,1,1,0,pass
9458,4,3,1,4,0,1,1,1,1359,130.0,106.0,0.722222,3.815385,0,23,3,0.400383,1,1,0,poor


In [25]:
print(df.X.shape) #confirm shape and make sure the column has bene dropped
print(df.X.results)


(9460, 22)
0       pass
1       pass
2       poor
3       fair
4       poor
        ... 
9455    poor
9456    poor
9457    pass
9458    poor
9459    poor
Name: results, Length: 9460, dtype: object


In [24]:
X = df.X.loc[ : , df.columns != 'results']
y = df.X[['results']]
#split 70% train 30% test
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=700, test_size=0.30)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(6622, 21) (6622, 1) (2838, 21) (2838, 1)


# Logistic Regression Model

# Naive Bayes Model